In [1]:
%store -r normalized_df

In [2]:
%matplotlib inline
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.preprocessing import LabelEncoder

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [3]:
X = normalized_df.drop(['kreftform'], axis=1)
y = normalized_df[['kreftform']]

In [18]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn import preprocessing
from sklearn import utils

data_dmatrix = xgb.DMatrix(data=X,label=y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

lab_enc = preprocessing.LabelEncoder()
encoded_y_train = lab_enc.fit_transform(y_train.values.ravel())
encoded_y_test = lab_enc.fit_transform(y_test.values.ravel())

xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 100)

xg_reg.fit(X_train,encoded_y_train)

y_pred = xg_reg.predict(X_test)

rmse = np.sqrt(mean_squared_error(encoded_y_test, y_pred))
print("RMSE: %f" % (rmse))

predictions = [round(value) for value in y_pred]
accuracy = accuracy_score(encoded_y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

RMSE: 0.349218
Accuracy: 85.78%


In [25]:
from sklearn.datasets import load_iris
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
from sklearn import utils

#X = normalized_df[['Sympt_blødning', 'Sympt_smerter', 'Sympt_ascites', 'Sympt_fatigue']]
#y = normalized_df[['kreftform']]

X = normalized_df.drop(['kreftform'], axis=1)
y = normalized_df[['kreftform']]

def run_once():
    
    data_dmatrix = xgb.DMatrix(data=X,label=y)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

    lab_enc = preprocessing.LabelEncoder()
    encoded_y_train = lab_enc.fit_transform(y_train.values.ravel())
    encoded_y_test = lab_enc.fit_transform(y_test.values.ravel())

    xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.3, learning_rate = 0.1,
                    max_depth = 5, alpha = 10, n_estimators = 100)

    xg_reg.fit(X_train,encoded_y_train)

    y_pred = xg_reg.predict(X_test)

    rmse = np.sqrt(mean_squared_error(encoded_y_test, y_pred))
    print("RMSE: %f" % (rmse))

    predictions = [round(value) for value in y_pred]
    accuracy = accuracy_score(encoded_y_test, predictions)
    print("Accuracy: %.2f%%" % (accuracy * 100.0))
    
def run_more_than_once(run_number):
    tot_acc = 0
    max_matrix = 0
    max_accuracy = 0

    min_matrix = 0
    min_accuracy = 1
    for i in range(run_number):
        
        data_dmatrix = xgb.DMatrix(data=X,label=y)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

        lab_enc = preprocessing.LabelEncoder()
        encoded_y_train = lab_enc.fit_transform(y_train.values.ravel())
        encoded_y_test = lab_enc.fit_transform(y_test.values.ravel())

        xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.3, learning_rate = 0.1,
                        max_depth = 5, alpha = 10, n_estimators = 100)

        xg_reg.fit(X_train,encoded_y_train)

        y_pred = xg_reg.predict(X_test)
        encoded_y_pred = lab_enc.fit_transform(y_pred)

        #rmse = np.sqrt(mean_squared_error(encoded_y_test, y_pred))
        #print("RMSE: %f" % (rmse))

        predictions = [round(value) for value in encoded_y_pred]
        accuracy = accuracy_score(encoded_y_test, predictions)
        #print("Accuracy: %.2f%%" % (accuracy * 100.0))

        accuracy = accuracy_score(encoded_y_test, predictions)

        if accuracy > max_accuracy:
            max_accuracy = accuracy
            max_matrix = confusion_matrix(encoded_y_test, encoded_y_pred)

        if accuracy < min_accuracy:
            min_accuracy = accuracy
            min_matrix = confusion_matrix(encoded_y_test, encoded_y_pred)

        tot_acc += accuracy
        tot = i+1
        mean = tot_acc/tot

    print('Maximum')
    print(max_accuracy, '\n', max_matrix)
    print('----------------------------------')
    print('Minimum')
    print(min_accuracy, '\n', min_matrix)
    print('----------------------------------')
    print('mean accuracy:', mean)

In [26]:
run_more_than_once(2)

Maximum
0.0044444444444444444 
 [[1 1 1 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 1 1 1]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
----------------------------------
Minimum
0.0044444444444444444 
 [[1 1 1 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 1 1 1]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
----------------------------------
mean accuracy: 0.0044444444444444444
